In [1]:
#=====[ Import dependencies ]====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import pickle
import os
from ai_trainer import Personal_Trainer

sys.path.append('data')
sys.path.append('squat_pushupData_10to20')

%matplotlib inline

In [2]:
pt = Personal_Trainer('NeckY')
pt.load_squats('multipleClass3.p')

In [3]:
squats = pt.get_squats()
labels = pt.get_labels()
squat = squats[0]

In [4]:
#======[ Returns index to frame with minimum y-coord for specified key ]=====
def get_min(squat,key):   
    
    #=====[ Return max because of inverse frame of reference of kinect ]=====
    return max([(coord,index) for index, coord in enumerate(squat[key])])[1]

#=====[ Returns index to frame with y-coord closes to the midpoint between start/end and squat position for specified key ]=====
def get_midpoint(squat,start,key):
    
    #=====[ Decide whether getting midpoint between start and squat or squat and end ]=====
    if start:
        start = 1
        end = get_min(squat,key)
    else:
        start = get_min(squat,key)
        end = squat.shape[0] - 1
        
    #=====[ Uses the 'true_mid' as an evaluation metric to find optimal index  ]=====
    true_mid = (squat.iloc[end][key] - squat.iloc[start][key])/2
    deltas = [(np.abs(true_mid - (squat.iloc[end][key] - squat.iloc[index][key])), index) for index in range(start,end)]
    return min(deltas)[1]

#=====[ Returns squat at the first position ]=====
def starting_position(squat):
    return squat.loc[1]

#=====[ Returns index to frame with y-coord closes to the midpoint between start and squat position for specified key ]=====
def start_to_squat(squat,key):
    return squat.loc[get_midpoint(squat,start=1,key=key),:]

#=====[ Returns frame with minimum y-coord for specified key ]=====
def squat_position(squat,key):
    return squat.loc[get_min(squat,key),:]

#=====[ Returns index to frame with y-coord closes to the midpoint between squat position and end for specified key ]=====
def squat_to_end(squat,key):
    return squat.loc[get_midpoint(squat,start=0,key=key),:]

def get_states(squat, key):
    
    states = []
    states.append(starting_position(squat))
    states.append(start_to_squat(squat,key))
    states.append(squat_position(squat,key))
    states.append(squat_to_end(squat,key))
    
    return states


In [128]:
def extract_stance_straightness(states):
    
    # Left heels directly under shoulders
    left_heels_under_shoulder =[state['AnkleLeftZ'] - state['ShoulderLeftZ'] for state in states]

    # Right heels directly under shoulders
    right_heels_under_shoulder = [state['AnkleRightZ'] - state['ShoulderRightZ'] for state in states]
    return np.concatenate([left_heels_under_shoulder, right_heels_under_shoulder],axis=1)

In [127]:
extract_stance_straightness(get_states(squat,'NeckY'))

array([  1.84719789e-04,  -1.50679753e-05,   6.18327868e-05,
        -2.32214511e-05,   1.41159098e-04,  -2.64732965e-06,
         7.12074593e-05,   1.49846906e-05])

In [129]:
extract_stance_straightness(get_states(squat,'NeckY'))

array([  1.84719789e-04,  -1.50679753e-05,   6.18327868e-05,
        -2.32214511e-05,   1.41159098e-04,  -2.64732965e-06,
         7.12074593e-05,   1.49846906e-05])

In [ ]:
def extract_stance_straightness(states):

    # W.R.T ALL 4 FRAMES
    left_heels_under_shoulder = []
    right_heels_under_shoulder = []
    for state in states:
        # Left heels directly under shoulders
        left_heels_under_shoulder.append(state['AnkleLeftZ'] - state['ShoulderLeftZ'])

        # Right heels directly under shoulders
        right_heels_under_shoulder.append(state['AnkleRightZ'] - state['ShoulderRightZ'])
    return np.concatenate([left_heels_under_shoulder, right_heels_under_shoulder],axis=1)

In [72]:
def extract_stance_shoulder_width(states):
    # W.R.T ALL 4 FRAMES
   
    left_heels_shoulder_apart = [state['AnkleLeftX'] - state['ShoulderLeftX']for state in states]

    right_heels_shoulder_apart = [state['AnkleRightX'] - state['ShoulderRightX'] for state in states]
    return np.concatenate([left_heels_shoulder_apart, right_heels_shoulder_apart],axis=1)

In [91]:
def extract_feet(states):

    # W.R.T ALL 4 FRAMES    
    left_feet_flat = []
    right_feet_flat = []
    for state in states:
        # Left foot flat on floor
        left_feet_flat.append(math.pow(state['KneeLeftZ'] - state['AnkleLeftZ'], 2))

        # Right foot flat on floor
        right_feet_flat.append(math.pow(state['KneeRightZ'] - state['AnkleRightZ'], 2))
    return np.concatenate([left_feet_flat, right_feet_flat],axis=1)

In [106]:
import math
def get_angle(state, joint1, joint2, joint3, axis1, axis2):
    # Left knee angle - formed by Y,Z coords W.R.T knee
    # P12
    bone1 = math.sqrt(math.pow(state[joint2 + axis1] - state[joint1 + axis1], 2) + math.pow(state[joint2 + axis2] - state[joint1 + axis2], 2))
    # P13
    bone2 = math.sqrt(math.pow(state[joint2 + axis1] - state[joint3 + axis1], 2) + math.pow(state[joint2 + axis2] - state[joint3 + axis2], 2))
    # P23
    distance = math.sqrt(math.pow(state[joint1 + axis1] - state[joint3 + axis1], 2) + math.pow(state[joint1 + axis2] - state[joint3 + axis2], 2))
    angle = math.acos((math.pow(bone1, 2) + math.pow(bone2, 2) - math.pow(distance, 2)) / (2 * bone1 * bone2))
    return angle

def get_angle_changes(angle1, angle2):

    assert(len(angle1) == len(angle2))
    
    full_angle1 = angle1[-1] - angle1[0]
    full_angle2 = angle2[-1] - angle2[0]
    
    ratios=[]
    
    for time in range(1,len(angle1)):
        ratios.append(abs(((angle1[time] - angle1[time-1]) / full_angle1) - (angle2[time] - angle2[time-1]) / full_angle2))
        
    return ratios


def bend_hips_knees(states):

    #left knee angle
    left_bend_knees = [get_angle(state, 'AnkleLeft','KneeLeft','HipLeft','Y','Z') for state in states]
    #left hip angle
    left_bend_hips = [get_angle(state,'SpineMid','HipLeft','KneeLeft','Y','Z') for state in states]
    #right knee angle
    right_bend_knees = [get_angle(state,'AnkleRight','KneeRight','HipRight','Y','Z') for state in states]
    #right hip angle
    right_bend_hips = [get_angle(state,'SpineMid','HipRight','KneeRight','Y','Z') for state in states]

    ratios = np.concatenate([get_angle_changes(left_bend_hips,left_bend_knees),get_angle_changes(right_bend_hips,right_bend_knees)],axis=1)
    
    return np.concatenate([left_bend_knees, left_bend_hips, right_bend_knees, right_bend_hips, ratios],axis=1)



In [107]:
def back_straight(states):

    back_angles = [get_angle(state,'SpineBase','SpineMid','SpineShoulder','Y','Z') for state in states]

    avg = np.average(back_angles)
    variance = map(lambda x : (x - avg)**2, back_angles)

    return np.concatenate([variance, avg], axis=1)


In [118]:
def head_aligned_back(states):

    head_angles = [get_angles(state,'Head','Neck','SpineShoulder','Y','Z') for state in states]

    avg = np.average(head_angles)
    variance = map(lambda x : (x - avg)**2, head_angles)
    
    return np.concatenate([variance, avg], axis=1)

In [126]:
def depth(states):
    
    state = max(states, key=lambda x: x['NeckY'])
   
    depth_angle = get_angle(state, 'AnkleLeft','KneeLeft','HipLeft','Y','Z')

    return np.array([depth_angle, state['HipLeftY'],state['HipRightY']])

In [ ]:
def back_hip_angle(states):

    slopes = []
    
    for state in states:
        slopes.append(abs(state['NeckY'] - np.average([state['HipLeftY'], state['HipRightY']])) / (state['NeckZ'] - np.average([state['HipLeftZ'], state['HipRightZ']])))
        
    return np.array(slopes)
